In [2]:
import pandas as pd
import numpy
import torch 
import numpy as np
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.append("..")
from utils import pad_collate
from dataloader_comma import CommaDataset
from dataloader_nuscenes import NUScenesDataset
from collections import Counter
from model import VTN
import matplotlib.pyplot as plt 
from PIL import Image
import glob
import os
from utils import * 
import re
from vis_utils import * 


In [24]:

for j, batch in enumerate(dataloader_nuscenes):
    if j < 5 : continue
    _, img, vego, angle, distance, m_lens, i_lens, s_lens, a_lens, d_lens = batch
    img, angle, distance, vego = img.to(gpu), angle.to(gpu), distance.to(gpu), vego.to(gpu)
    (logits, attns), concepts = model(img, angle, distance, vego)
    
    angle, distance, vego, logits = angle.to("cpu"), distance.to("cpu"), vego.to("cpu"), logits.detach().cpu().to("cpu")

    concepts
    atten = attns[0][:,:,0:concepts.shape[1]].detach()
    seq_len = atten.shape[2]
    alignment_array = get_aligned_attention(atten.squeeze().cpu(), seq_len)
    speed_graph_0 = alignment_array.sum(axis=0)[8:-8]
    speed_graph_0 = moving_average(speed_graph_0, 10)

    atten = attns[1][:,:,0:concepts.shape[1]].detach()
    seq_len = atten.shape[2]
    alignment_array = get_aligned_attention(atten.squeeze().cpu(), seq_len)
    speed_graph_1 = alignment_array.sum(axis=0)[8:-8]
    speed_graph_1 = moving_average(speed_graph_1, 10)

    atten = attns[2][:,:,0:concepts.shape[1]].detach()
    seq_len = atten.shape[2]
    alignment_array = get_aligned_attention(atten.squeeze().cpu(), seq_len)
    speed_graph_2 = alignment_array.sum(axis=0)[8:-8]
    speed_graph_2 = moving_average(speed_graph_2, 10)
    
    f = []
    inter = []
    m = (speed_graph_2 + speed_graph_1 + speed_graph_0)/3
    ali = 1 + m 
    print(ali.shape)
    concepts = concepts.squeeze()
    weighted_concepts = concepts * ali

    top5_indices = torch.tensor(concepts).topk(5).indices
    
    for i, elem0 in enumerate(top5_indices):
        inter = []
        for elem in top5_indices[max(i-5, 0):min(i+5,len(top5_indices))]:
            l = elem.cpu().numpy().tolist()
            if 131 in l:
                l.remove(131)
            inter.extend(l)
        count_dict = Counter(inter)
        # Get the top 5 most occurring numbers
        top_5 = count_dict.most_common(2)
        intermediate = []
        for a in top_5: 
            intermediate.append(scenarios[a[0]])
        f.append(intermediate)

    normalized_weights = atten#np.array(attention_weights) / np.sum(attention_weights)
    print(atten.shape)
    

    fig, ax = plt.subplots(figsize=(10, 6))
    plt.plot(speed_graph_0, label="1")
    plt.plot(speed_graph_1, label="2")
    plt.plot(speed_graph_2, label="3")
    plt.plot(m, label="mean")
    plt.plot(alignment_array[0:70, :].sum(axis=0), label="align")
    plt.legend()
    #heatmap = ax.pcolormesh(alignment_array[0:70, :], cmap='hot')
    # Add colorbar
    #cbar = plt.colorbar(heatmap)

    # Add labels and title
    ax.set_xlabel('Sequence Position')
    ax.set_ylabel('Window')
    ax.set_title('Longformer Sliding Chunk Attention')
    plt.show()
    # Show the plot
    #plt.savefig(f"/home/jessica/personalized_driving_toyota/attention_vis{j}.pdf")
    #plt.close()
    break

    

(27,)


TypeError: can't convert cuda:1 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [3]:
plt.rcParams.update({'font.size': 26}) 